<a href="https://colab.research.google.com/github/parimalakettymuthu/MachineLearning-Projects/blob/main/Parimala_Kettymuthu_HW7_task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [120]:
#Defined the auto reload
%load_ext autoreload  
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [121]:
#Imported the required packages and defined the path for accessing google drive files & custom-functions
import sys
if 'google.colab' in str(get_ipython()):
  from google.colab import drive
  drive.mount('/content/drive')
  
  basepath = '/content/drive/My Drive/NLP'
  sys.path.append('/content/drive/My Drive/NLP/custom-functions')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [122]:
#Installing the gensim package
!pip install --upgrade gensim -qq


In [123]:
#Checking the system path
sys.path

['/content',
 '/env/python',
 '/usr/lib/python39.zip',
 '/usr/lib/python3.9',
 '/usr/lib/python3.9/lib-dynload',
 '',
 '/usr/local/lib/python3.9/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.9/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/content/drive/My Drive/NLP/custom-functions',
 '/content/drive/My Drive/NLP/custom-functions',
 '/content/drive/My Drive/NLP/custom-functions',
 '/content/drive/My Drive/NLP/custom-functions']

In [124]:
#Importing the required packages
from pathlib import Path
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, accuracy_score, hamming_loss, multilabel_confusion_matrix, f1_score
import matplotlib.pyplot as plt

In [125]:
#importing classes from custom packages
import custom_preprocessor_mod as cp
from plot_learning_curve import plot_learning_curve as plt


In [126]:
#Importing the required packages
import gensim
from gensim.models import KeyedVectors
import joblib

In [127]:
#Checking the genism version
gensim.__version__

'4.3.1'

In [128]:
#Defined the google drive folders for accessing/saving models related files
embeddings_folder = Path(basepath)/ 'assignment7/WordEmbeddings'
data_folder = Path(basepath)/ 'assignment7/MultiLabel_Classification'
model_saving_folder = Path(basepath)/ 'assignment7/MultiLabel_Classification'

Gensim Vectorizer

In [129]:
from gensim_vectorizer  import GensimVectorizer  #Imported the GensimVectorizer

In [130]:
import gensim.downloader as loader
#w2vec_google_news = loader.load("word2vec-google-news-300")
#!unzip '/content/drive/MyDrive/data/WordEmbeddings/wiki-news-300d-1M-subword.vec.zip' -d data_folder

In [131]:
#Accessing the folder path for dataset file and loading the contents into local variable
data_cleaned = data_folder/ "df_multilabel_hw_cleaned.joblib"
stackExchange_dataset = joblib.load(data_cleaned)

In [132]:
type(stackExchange_dataset)

pandas.core.frame.DataFrame

In [133]:
#Checking the dataset shape
stackExchange_dataset.shape

(47427, 3)

In [134]:
#Checking the dataset columns
stackExchange_dataset.columns

Index(['cleaned_text', 'Tags', 'Tag_Number'], dtype='object')

In [135]:
#Checking the dataset 1st few observations
stackExchange_dataset.head(5)

,cleaned_text,Tags,Tag_Number
0,asp query stre dropdown webpage follow control...,c# asp.net,"[0, 9]"
1,run javascript code server java code want run ...,java javascript,"[1, 3]"
2,linq sql throw exception row find change hi li...,c# asp.net,"[0, 9]"
3,run python script php server run nginx web ser...,php python,"[2, 7]"
4,advice write function m try write function res...,javascript jquery,"[3, 5]"


In [136]:
#Accessing the dataset values and assigning it to X & y
X = stackExchange_dataset['cleaned_text'].values
y = stackExchange_dataset['Tag_Number'].values
print(type(X), type(y))
print(X.dtype, y.dtype)

<class 'numpy.ndarray'> <class 'numpy.ndarray'>
object object


In [137]:
#Installing the swifter packages
!pip install swifter -qq

In [138]:
#Importing the required packages
import swifter
import ast
stackExchange_dataset['Tag_Number_list'] = stackExchange_dataset['Tag_Number'].swifter.apply(lambda x: ast.literal_eval(x))

Pandas Apply:   0%|          | 0/47427 [00:00<?, ?it/s]

In [139]:
stackExchange_dataset['Tag_Number_list'][0]   #Checking the contents of 1st observation of the Tag_Number_list

[0, 9]

In [140]:
y_final = stackExchange_dataset['Tag_Number_list'].values  #Assign the Tag_Number_list field values to y variable

In [141]:
#Performing the preprocessing files 
from sklearn.preprocessing import MultiLabelBinarizer as mlb
y_stackExchange_encoding = mlb().fit_transform(y_final)

In [142]:
type(y_stackExchange_encoding)  #Checking the type of encoding variable

numpy.ndarray

In [143]:
y_stackExchange_encoding.shape #Checking the shape of encoding variable

(47427, 10)

In [144]:
#Performing data splitting for train, test & valid
from sklearn.model_selection import train_test_split
X_sExchange_train, X_valid_test, y_sExchange_train, y_valid_test = train_test_split(X, y_stackExchange_encoding, test_size=0.4, random_state=42)
X_sExchange_valid, X_sExchange_test, y_sExchange_valid, y_sExchange_test = train_test_split(X_valid_test, y_valid_test, test_size=0.5, random_state=42)

In [145]:
#Checking the length of X (train, test & valid)
print(len(X_sExchange_train), len(X_sExchange_test), len(X_sExchange_valid))

28456 9486 9485


In [146]:
#Checking the shape of y (train, test & valid)
print(y_sExchange_train.shape, y_sExchange_test.shape, y_sExchange_valid.shape)

(28456, 10) (9486, 10) (9485, 10)


In [147]:
#Accessing the pre-trained vectors generated file and loading into local variable
pretrained_sExchange_file = str(embeddings_folder/ "model_stackExchange_CBOW.bin")
sExchange_vectors = KeyedVectors.load(pretrained_sExchange_file)


In [148]:
type(sExchange_vectors)  #Checking the type of the local vector variable

gensim.models.keyedvectors.KeyedVectors

In [149]:
#!pip install OneVsRestClassifier

In [150]:
#pipeline using OneVsRestClassifier (LogisticRegression) & GensimVectorizer for multilabel 
from sklearn.multiclass import OneVsRestClassifier 
se_pipeline=Pipeline(
    [
      ("vectorizer", GensimVectorizer(sExchange_vectors)),
      ("clf", OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=1))
    ]
)

Hyperparameter Tuning

In [151]:
se_pipeline.fit(X_sExchange_train, y_sExchange_train)  #Fitted the train dataset

Pipeline(steps=[('vectorizer',
                 GensimVectorizer(pretrained_vectors=<gensim.models.keyedvectors.KeyedVectors object at 0x7fd79ccee2b0>)),
                ('clf',
                 OneVsRestClassifier(estimator=LogisticRegression(solver='sag'),
                                     n_jobs=1))])

In [152]:
#Predicted using train,test & valid dataset
svc_train_pred = se_pipeline.predict(X_sExchange_train)
svc_test_pred = se_pipeline.predict(X_sExchange_test) 
svc_valid_pred = se_pipeline.predict(X_sExchange_valid)

In [153]:
#Checking the hamming loss for train, test & valid dataset
print("Training Hamming loss: ", hamming_loss(y_sExchange_train, svc_train_pred))
print("Test Hamming loss: ", hamming_loss(y_sExchange_test, svc_test_pred))
print("Valid Hamming loss: ", hamming_loss(y_sExchange_valid, svc_valid_pred))

Training Hamming loss:  0.0593864211414113
Test Hamming loss:  0.06317731393632722
Valid Hamming loss:  0.0615392725355825


In [154]:
#Checking the F1 score for train, test & valid dataset
print("Training F1 Score: ", f1_score(y_sExchange_train, svc_train_pred, average="micro")*100)
print("Test F1 Score: ", f1_score(y_sExchange_test, svc_test_pred, average="micro")*100)
print("Valid F1 Score: ", f1_score(y_sExchange_valid, svc_valid_pred, average="micro")*100)

Training F1 Score:  84.9854732521257
Test F1 Score:  83.95405499477897
Valid F1 Score:  84.38010115336242


In [155]:
#Defined pipeline using the LinearSVC with GensimVectorizer 
from sklearn.svm import LinearSVC
se_svc_pipeline = Pipeline([
    ('vectorizer', GensimVectorizer(sExchange_vectors)),
    ('clf', OneVsRestClassifier(LinearSVC(), n_jobs=1))
])

In [156]:
se_svc_pipeline.fit(X_sExchange_train, y_sExchange_train)   #Fitted the train dataset

Pipeline(steps=[('vectorizer',
                 GensimVectorizer(pretrained_vectors=<gensim.models.keyedvectors.KeyedVectors object at 0x7fd79ccee2b0>)),
                ('clf', OneVsRestClassifier(estimator=LinearSVC(), n_jobs=1))])

In [157]:
#Predicted using train, test & valid dataset
svc_train_pred = se_svc_pipeline.predict(X_sExchange_train)
svc_test_pred = se_svc_pipeline.predict(X_sExchange_test) 
svc_valid_pred = se_svc_pipeline.predict(X_sExchange_valid)

In [158]:
#Calculating the hamming loss using train, test & valid dataset
print("Training Hamming loss: ", hamming_loss(y_sExchange_train, svc_train_pred))
print("Test Hamming loss: ", hamming_loss(y_sExchange_test, svc_test_pred))
print("Valid Hamming loss: ", hamming_loss(y_sExchange_valid, svc_valid_pred))

Training Hamming loss:  0.048773545122294065
Test Hamming loss:  0.052804132405650434
Valid Hamming loss:  0.05150237216657881


In [159]:
#Calculating the F1 score using train, test & valid dataset
print("Training F1 Score: ", f1_score(y_sExchange_train, svc_train_pred, average="micro")*100)
print("Test F1 Score: ", f1_score(y_sExchange_test, svc_test_pred, average="micro")*100)
print("Valid F1 Score: ", f1_score(y_sExchange_valid, svc_valid_pred, average="micro")*100)

Training F1 Score:  87.95863301550395
Test F1 Score:  86.91107685071468
Valid F1 Score:  87.25906992514541


In [159]:
"""
Logistic Regression based multilabel pipeline based results:
Training Hamming loss:  0.0593864211414113
Test Hamming loss:  0.06317731393632722
Valid Hamming loss:  0.0615392725355825
Training F1 Score:  84.9854732521257
Test F1 Score:  83.95405499477897
Valid F1 Score:  84.38010115336242

LinearSVC based multilabel pipeline based results:
Training Hamming loss:  0.048773545122294065
Test Hamming loss:  0.052804132405650434
Valid Hamming loss:  0.05150237216657881
Training F1 Score:  87.95863301550395
Test F1 Score:  86.91107685071468
Valid F1 Score:  87.25906992514541

Based on these two methods, LinearSVC model have performed better with lower hamming loss and higher F1 score; Further this 
model can be suggested for business. 
"""